In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

In [6]:
#nhập dữ liệu
df_softmax = pd.read_csv("data.csv", delimiter = ";")
pd.set_option('display.width', 500)
df_softmax.columns = df_softmax.columns.str.strip()

# Encoding cho biến target thành ma trận one-hot
def encode_target(value: str) -> list:
    if value == "Dropout":
        return [1, 0, 0]
    elif value == "Enrolled":
        return [0, 1, 0]
    else:
        return [0, 0, 1]

# Chuyển đổi 'Target' thành danh sách one-hot
df_softmax["Target"] = df_softmax["Target"].apply(encode_target)

# Tạo dataframe mới từ danh sách one-hot
y_one_hot = pd.DataFrame(df_softmax["Target"].tolist(), 
                         columns=["Dropout", "Enrolled", "Graduate"])  

# Gộp lại với df_softmax (bỏ cột Target cũ)
df_softmax = pd.concat([df_softmax.drop(columns=["Target"]), y_one_hot], axis=1)

In [7]:
# Xác định X và y
X_softmax = df_softmax.drop(columns=["Dropout", "Enrolled", "Graduate"]).values
y_softmax = df_softmax[["Dropout", "Enrolled", "Graduate"]].values

# Chia dataset thành 3 phần theo tỉ lệ 8:1:1
X_train_s, X_temp_s, y_train_s, y_temp_s = train_test_split(X_softmax, y_softmax, test_size=0.2, random_state=42)
X_dev_s, X_test_s, y_dev_s, y_test_s = train_test_split(X_temp_s, y_temp_s, test_size=0.5, random_state=42)


In [8]:
# Chuẩn hóa dữ liệu
scaler_s = StandardScaler()
X_train_s = scaler_s.fit_transform(X_train_s)
X_dev_s = scaler_s.transform(X_dev_s)
X_test_s = scaler_s.transform(X_test_s)

# Softmax Regression Class
class SoftmaxRegression:
    def __init__(self, epochs: int, lr: float) -> None:
        self.lr = lr
        self.epochs = epochs
        self.losses = []
        self.metric = []
        self.theta = None  # Định nghĩa sẵn để tránh lỗi khi gọi predict() trước fit()

    def softmax(self, z: np.ndarray) -> np.ndarray:
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Tránh overflow
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def loss_fn(self, y: np.ndarray, y_hat: np.ndarray) -> float:
        y_hat = np.clip(y_hat, 1e-8, 1 - 1e-8)  # Tránh log(0)
        return -np.mean(np.sum(y * np.log(y_hat), axis=1))

    def accuracy(self, y: np.ndarray, y_hat: np.ndarray) -> float:
        y_pred_labels = np.argmax(y_hat, axis=1)
        y_true_labels = np.argmax(y, axis=1)  # Giả định y là one-hot
        return np.mean(y_pred_labels == y_true_labels)

    def predict(self, X: np.ndarray) -> np.ndarray:
        if self.theta is None:
            raise ValueError("Model chưa được train! Hãy gọi fit() trước khi predict().")
        z = np.dot(X, self.theta)
        return self.softmax(z)  # Trả về xác suất của 3 lớp

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        n, d = X.shape
        k = y.shape[1]  # Số lớp
        self.theta = np.random.randn(d, k) * 0.01  # Khởi tạo nhỏ để tránh gradient lớn

        with tqdm.tqdm(range(self.epochs)) as pb:
            for e in pb:
                pb.set_description(f"Epoch {e + 1}")
                y_hat = self.predict(X)  # Xác suất softmax
                gradient = (1/n) * np.dot(X.T, (y_hat - y))  # Gradient descent
                self.theta -= self.lr * gradient  # Cập nhật tham số

                loss = self.loss_fn(y, y_hat)
                acc = self.accuracy(y, y_hat)
                pb.set_postfix({"loss": loss, "acc": acc})

                self.losses.append(loss)
                self.metric.append(acc)

In [9]:
# Huấn luyện mô hình
model_s = SoftmaxRegression(lr=0.01, epochs=1000)
model_s.fit(X_train_s, y_train_s)

# Kiểm tra trên tập dev
y_pred_dev = model_s.predict(X_dev_s)

# Nếu y_dev_s đã là vector nhãn (0,1,2), không cần np.argmax()
if y_dev_s.ndim == 1:
    y_dev_labels = y_dev_s
else:
    y_dev_labels = np.argmax(y_dev_s, axis=1)

y_pred_dev_labels = np.argmax(y_pred_dev, axis=1)

accuracy_dev = np.mean(y_pred_dev_labels == y_dev_labels)
print(f"Accuracy trên tập dev: {accuracy_dev:.2f}")

# Kiểm tra trên tập test
y_pred_test_s = model_s.predict(X_test_s)

# Nếu y_test_s đã là nhãn số, không cần np.argmax()
if y_test_s.ndim == 1:
    y_test_labels_s = y_test_s
else:
    y_test_labels_s = np.argmax(y_test_s, axis=1)

y_pred_test_labels_s = np.argmax(y_pred_test_s, axis=1)

accuracy_test_s = np.mean(y_pred_test_labels_s == y_test_labels_s)
print(f"Accuracy trên tập test: {accuracy_test_s:.2f}")

# Confusion Matrix
cm = confusion_matrix(y_dev_labels, y_pred_dev_labels)
print("Confusion Matrix:\n", cm)

precision = precision_score(y_dev_labels, y_pred_dev_labels, average="macro")
recall = recall_score(y_dev_labels, y_pred_dev_labels, average="macro")
f1 = f1_score(y_dev_labels, y_pred_dev_labels, average="macro")

print(f"Precision (Macro): {precision:.2f}")
print(f"Recall (Macro): {recall:.2f}")
print(f"F1-score (Macro): {f1:.2f}")

Epoch 1000: 100%|██████████| 1000/1000 [00:03<00:00, 278.02it/s, loss=0.602, acc=0.761]


Accuracy trên tập dev: 0.76
Accuracy trên tập test: 0.74
Confusion Matrix:
 [[124  21  16]
 [ 15  20  33]
 [ 12  11 190]]
Precision (Macro): 0.67
Recall (Macro): 0.65
F1-score (Macro): 0.66
